In [13]:
# Импортируем все необходимые библиотеки

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split, KFold
from catboost import CatBoostRegressor, Pool, CatBoostClassifier
from catboost.utils import eval_metric
from sklearn.datasets import make_multilabel_classification

In [14]:
# Открываем датасет с разбивкой по табуляциям

df = pd.read_csv('../data/marketing_campaign.csv', sep='\t')

In [15]:
# Смотрим на соотношение значений подозрительных фичей, чтобы понять, какие из этих фичей стоит отбросить.

print(df['Complain'].value_counts())
print(df['Z_CostContact'].value_counts())
print(df['Z_Revenue'].value_counts())
print(df['Response'].value_counts())

Complain
0    2219
1      21
Name: count, dtype: int64
Z_CostContact
3    2240
Name: count, dtype: int64
Z_Revenue
11    2240
Name: count, dtype: int64
Response
0    1906
1     334
Name: count, dtype: int64


In [16]:
# Выделяем категориальные признаки (переводить их в int не требуется, так как CatBoost умеет работать
# с категориальными фичами)
cat_features = ['Education', 'Marital_Status']

# Отбираем таргеты (в нашем случае это группа фичей Promotion)
targets = ['AcceptedCmp1', 'AcceptedCmp2', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5']

# Удаляем бесполезные фичи, которые мы определили ранее
features2drop = ['ID', 'Z_Revenue', 'Z_CostContact'] 

# Заменяем null'ы (если есть) в категориальных фичах на пустые строки
for c in cat_features:
    df[c] = df[c].astype(str)

# Переводим фичу 'Dt_Customer' (дату регистрации клиента в компании) в int, оставляя только год 
df['Dt_Customer'] = df['Dt_Customer'].apply(lambda x: x.split('-')[-1])

# Записываем в список результат из отобранных для обучения фичей
filtered_features = [i for i in df.columns if (i not in targets and i not in features2drop)]

In [56]:
# Выделяем X (фичи) и y (таргеты) из нашего датасета 
X = df[filtered_features].drop(targets, axis=1, errors="ignore")
y = df[targets]

# Разбиваем наш датасет на 2 части: train (80%) и test (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Проводим предварительную квантилизацию выборок для ускорения обучения модели CatBoost
train_pool = Pool(X_train, y_train, cat_features=cat_features)
test_pool = Pool(X_test, y_test, cat_features=cat_features)

In [65]:
# В качестве модели был выбран градиентный бустинг CatBoost, так как он даёт хорошие результаты
# на дефолтных параметрах, а также умеет работать с категориальными фичами

# Инициализируем CatBoost класифаер
clf = CatBoostClassifier(
    # В качестве функции потерь берём MultiLogloss (так как он лучше всего подходит 
    # для нашей задачи - MultiLabel классификации)
    loss_function='MultiLogloss',
    
    # В качестве метрики оценки модели выбираем HammingLoss (так как он лучше всего подходит 
    # для нашей задачи - MultiLabel классификации)
    eval_metric='HammingLoss',
    
    # Для максимального количества деревьев берём число 500
    iterations=500,
    
    # Указываем таргеты
    class_names=targets,
    
    # Фиксируем random_state для воспроизводимости результатов
    random_state=42
)

# Обучаем модель с test_pool в качестве eval-сета, а также используем лучшую модель 
# и задаём early_stopping на 100 итераций
clf.fit(train_pool, eval_set=test_pool, plot=True, verbose=50, use_best_model=True, early_stopping_rounds=100)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.0366
0:	learn: 0.0582589	test: 0.0611607	best: 0.0611607 (0)	total: 14.3ms	remaining: 14.3s
50:	learn: 0.0493304	test: 0.0593750	best: 0.0575893 (35)	total: 521ms	remaining: 9.69s
100:	learn: 0.0375000	test: 0.0562500	best: 0.0562500 (99)	total: 1.18s	remaining: 10.5s
150:	learn: 0.0290179	test: 0.0566964	best: 0.0553571 (138)	total: 1.76s	remaining: 9.88s
200:	learn: 0.0233259	test: 0.0562500	best: 0.0553571 (138)	total: 2.33s	remaining: 9.28s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.05535714286
bestIteration = 138

Shrink model to first 139 iterations.


In [66]:
# Предсказываем, с какой попытки клиент примет предложение
test_predict = clf.predict(X_test)

# Вычисляем точность модели на тестовой выборке - получаем точность 0.7901785714285714 (весьма неплохой результат)
# Его также можно улучшить с помощью feature selection'а и feature engineering'а (но у меня не получилось :/)
accuracy = eval_metric(y_test.to_numpy(), test_predict, 'Accuracy')[0]
print(f'Accuracy: {accuracy}')

Accuracy: 0.7901785714285714


In [64]:
# Проверям feature importance, чтобы посмотреть, что мы можем использовать для feature selection
clf.get_feature_importance(prettified=True)

,Feature Id,Importances
0,Education,12.674766
1,MntWines,11.390720
2,Marital_Status,8.459371
3,NumStorePurchases,7.533881
4,Income,7.277208
5,Response,6.703975
6,MntGoldProds,6.069284
7,Dt_Customer,4.789835
8,MntMeatProducts,4.494427
9,MntFishProducts,4.158689


In [4]:
# Анализируем датасет
df

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,04-09-2012,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,08-03-2014,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,21-08-2013,26,426,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,10-02-2014,26,11,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,19-01-2014,94,173,...,5,0,0,0,0,0,0,3,11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,10870,1967,Graduation,Married,61223.0,0,1,13-06-2013,46,709,...,5,0,0,0,0,0,0,3,11,0
2236,4001,1946,PhD,Together,64014.0,2,1,10-06-2014,56,406,...,7,0,0,0,1,0,0,3,11,0
2237,7270,1981,Graduation,Divorced,56981.0,0,0,25-01-2014,91,908,...,6,0,1,0,0,0,0,3,11,0
2238,8235,1956,Master,Together,69245.0,0,1,24-01-2014,8,428,...,3,0,0,0,0,0,0,3,11,0


In [5]:
# Смотрим все доступные колонки
df.columns

Index(['ID', 'Year_Birth', 'Education', 'Marital_Status', 'Income', 'Kidhome',
       'Teenhome', 'Dt_Customer', 'Recency', 'MntWines', 'MntFruits',
       'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts',
       'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases',
       'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth',
       'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1',
       'AcceptedCmp2', 'Complain', 'Z_CostContact', 'Z_Revenue', 'Response'],
      dtype='object')